want to compare the increase of cabi rides between maryland and dc

2022,2023,2024

steps
filter the data set for stations and rides in dc and maryland:
1) take the complete dataset, turn it into geodataframe
2) join it with maryland gdf
3) make plot to compare the difference of growth between them

dive deeper into maryland / prince george county for 2023 -2024

In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
from geopy import distance
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from branca.colormap import LinearColormap
from collections import Counter
import json
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import shape

In [2]:
data_types = {
    "rideable_type": "category", 
    "start_station_name": "category", 
    "end_station_name": "category", 
    "member_casual":"category",
    # "ride_id":"uint32",
    "time_of_day":"category",
    "trip_type":"category"}

In [3]:
bikes = pd.read_csv("bikes_clean.csv",parse_dates= ["started_at", "ended_at"], dtype=data_types,low_memory=False)

In [4]:
cabi_2024 = pd.read_csv("cabi_2024.csv",parse_dates= ["started_at", "ended_at"], low_memory=False)

In [5]:
cabi_updated = pd.concat([bikes, cabi_2024],ignore_index=True,axis=0)

In [6]:
cabi_updated.shape

(16808320, 20)

In [7]:
cabi_updated =cabi_updated.drop(columns = ["ride_id","start_station_id","end_station_id","start_time","end_time","start_hour","end_hour","Unnamed: 0"])

In [8]:
cabi_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 12 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   rideable_type          object  
 1   started_at             object  
 2   ended_at               object  
 3   start_station_name     object  
 4   end_station_name       object  
 5   member_casual          object  
 6   start_lat              float64 
 7   start_lng              float64 
 8   end_lat                float64 
 9   end_lng                float64 
 10  trip_duration_minutes  float64 
 11  time_of_day            category
dtypes: category(1), float64(5), object(6)
memory usage: 1.4+ GB


In [9]:
cabi_updated["rideable_type"] = cabi_updated["rideable_type"].astype("category")
cabi_updated["start_station_name"] = cabi_updated["start_station_name"].astype("category")
cabi_updated["end_station_name"] = cabi_updated["end_station_name"].astype("category")
cabi_updated["member_casual"] = cabi_updated["member_casual"].astype("category")


In [10]:
cabi_updated['started_at'] = pd.to_datetime(cabi_updated['started_at'])
cabi_updated['year'] = cabi_updated['started_at'].dt.year

In [11]:
# selecting only 2022,2023,2024
cabi_updated_2022_to_2024 = cabi_updated[(cabi_updated["year"] == 2022)| (cabi_updated["year"] == 2023)|(cabi_updated["year"] == 2024)]

In [12]:
cabi_updated_2022_to_2024['started_at'] = cabi_updated_2022_to_2024['started_at'].astype("object")

/var/folders/kj/_mqw0wd165b_qz356cmtmkzc0000gn/T/ipykernel_13151/295875490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabi_updated_2022_to_2024['started_at'] = cabi_updated_2022_to_2024['started_at'].astype("object")


In [13]:
cabi_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   rideable_type          category      
 1   started_at             datetime64[ns]
 2   ended_at               object        
 3   start_station_name     category      
 4   end_station_name       category      
 5   member_casual          category      
 6   start_lat              float64       
 7   start_lng              float64       
 8   end_lat                float64       
 9   end_lng                float64       
 10  trip_duration_minutes  float64       
 11  time_of_day            category      
 12  year                   int32         
dtypes: category(5), datetime64[ns](1), float64(5), int32(1), object(1)
memory usage: 1.0+ GB


In [ ]:
# # Create a GeoDataFrame for the starting points
geometry = [Point(xy) for xy in zip(cabi_updated['start_lng'], cabi_updated['start_lat'])]
cabi_updated_gdf = gpd.GeoDataFrame(cabi_updated, geometry=geometry, crs="EPSG:4326")



In [49]:
# geometry for starting unique stations
avg_lat_per_station = cabi_updated.groupby("start_station_name", as_index=False, observed=False)["start_lat"].mean()
avg_lng_per_station = cabi_updated.groupby("start_station_name", as_index=False, observed = False)["start_lng"].mean()

cabi_unique_stations = avg_lat_per_station.merge(avg_lng_per_station)

geometry = [Point(xy) for xy in zip(cabi_unique_stations['start_lng'], cabi_unique_stations['start_lat'])]
cabi_unique_stations_gdf = gpd.GeoDataFrame(cabi_unique_stations, geometry=geometry, crs="EPSG:4326")

In [37]:
bikes["start_station_name"].nunique()

860

In [38]:
cabi_2024["start_station_name"].nunique()

818

In [39]:
cabi_updated["start_station_name"].nunique()

914

In [15]:
cabi_updated_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   rideable_type          category      
 1   started_at             datetime64[ns]
 2   ended_at               object        
 3   start_station_name     category      
 4   end_station_name       category      
 5   member_casual          category      
 6   start_lat              float64       
 7   start_lng              float64       
 8   end_lat                float64       
 9   end_lng                float64       
 10  trip_duration_minutes  float64       
 11  time_of_day            category      
 12  year                   int32         
 13  geometry               geometry      
dtypes: category(5), datetime64[ns](1), float64(5), geometry(1), int32(1), object(1)
memory usage: 1.2+ GB


In [16]:
# cabi_updated_gdf['started_at'] = pd.to_datetime(cabi_updated_gdf['started_at'])#, format='%Y-%m-%d %H:%M:%S')
# cabi_updated_gdf['ended_at'] = pd.to_datetime(cabi_updated_gdf['ended_at'])
# cabi_updated_gdf['year'] = cabi_updated_gdf['started_at'].dt.year
# cabi_updated_gdf['month'] = cabi_updated_gdf['started_at'].dt.month

In [17]:
# cabi_updated_gdf.to_csv("cabi_updated_gdf.csv")

# sample

In [18]:
# # cabi updated gdf sample 

# # 108.000 rows sample (3000 per month)
# samples = []

# # Loop over years and months to sample 1,000 rows for each month
# for year in range(2021, 2025):  # Covering 2021, 2022, and 2024
#     for month in range(1, 13):  # Months from 1 to 12
#         # Filter for the specific year and month
#         filtered_data = cabi_updated_gdf[(cabi_updated_gdf["year"] == year) & (cabi_updated_gdf["month"] == month)]
        
#         # Sample 1,000 rows if there are enough rows available
#         if len(filtered_data) >= 3000:
#             sample = filtered_data.sample(n=3000, random_state=42)
#         else:  # If less than 3,000 rows, take all available rows
#             sample = filtered_data
            
#         samples.append(sample)

# # Combine all the samples into a single DataFrame
# bikes_monthly_sample_3000 = pd.concat(samples, ignore_index=True)

In [19]:
# bikes_monthly_sample_3000.to_csv("bikes_sample_updated.csv", index=False)

In [20]:
# uploading ward info so I can filter out DC area
with open ("Wards_from_2022.geojson") as i:
    wards = json.loads(i.read())

features =wards["features"]

ward_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326" )

In [21]:
ward_gdf = ward_gdf[["WARD","NAME","geometry"]]

In [22]:
dc_wards_gdf = gpd.sjoin(cabi_updated_gdf, ward_gdf, how="left", predicate="within")

In [ ]:
#maryland boundaries
with open ("Maryland_Physical_Boundaries_-_County_Boundaries_(Detailed).geojson") as i:
    maryland = json.loads(i.read())

features = maryland["features"]

#GDF
maryland_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326")
    

In [ ]:
# performing spatial join between the UNIQUE bike stations and maryland shape
cabi_maryland_unique_gdf = gpd.sjoin(cabi_unique_stations_gdf, maryland_gdf, how="left", predicate="within")

In [51]:
prince_george_unique = cabi_maryland_unique_gdf[cabi_maryland_unique_gdf["COUNTY"] == "Prince George's"]

In [52]:
prince_george_unique["start_station_name"].nunique()

35

In [55]:
list(prince_george_unique["start_station_name"].unique())

['1301 McCormick Dr / Wayne K. Curry Admin Bldg',
 '40th Ave & Bladensburg Rd',
 'Baltimore Ave & Jefferson St',
 'Baltimore Ave & Van Buren St / Riverdale Park Station',
 'Baltimore Avenue and Hotel Drive at UMD',
 'Bladensburg Waterfront Park',
 'Bowdoin Ave & Calvert Rd/ College Park Metro',
 'Bowdoin Ave & Calvert Rd/ College Park Station',
 'Capitol Heights Metro',
 'Chillum Rd & Riggs Rd / Riggs Plaza',
 'Crescent Rd & Ridge Rd',
 'Fleet St & Waterfront St',
 'Greenbelt Station Parkway',
 'Guilford Drive & Rowalt Drive / UMD',
 'Hyattsville Library / Adelphi Rd & Toledo Rd',
 "Largo Rd & Campus Way / Prince Georges's Comm Col",
 'Largo Town Center Metro',
 'National Harbor Carousel',
 'New Hampshire Ave & East-West Hwy',
 'Northwestern High School',
 'Oglethorpe St & 42nd Ave',
 'Oxon Hill Park & Ride',
 'Perry & 35th St',
 "Prince George's Plaza Metro",
 'Queens Chapel & Hamilton St',
 'Rhode Island Ave & 39th St / Brentwood Arts Exchange',
 'Rhode Island Avenue /Charles Armentr

#REVIEW - 35 unique stations from 2021 to 2024. This makes more sense than 90. I had to determine the unique stations first, before doing the join

In [ ]:
#REVIEW - when doing the sjoin with all the starting points (not only stations), many stations end up incorrectly sorted.
cabi_maryland_gdf = gpd.sjoin(cabi_updated_gdf, maryland_gdf, how="left", predicate="within")

In [31]:
prince_george = cabi_maryland_gdf[cabi_maryland_gdf["COUNTY"] == "Prince George's"]

In [47]:
prince_george_2024 = prince_george[prince_george["year"]==2024]

In [36]:
prince_george["start_station_name"].nunique()

91

In [61]:
prince_george_fixed = prince_george[prince_george["start_station_name"].isin(['1301 McCormick Dr / Wayne K. Curry Admin Bldg',
 '40th Ave & Bladensburg Rd',
 'Baltimore Ave & Jefferson St',
 'Baltimore Ave & Van Buren St / Riverdale Park Station',
 'Baltimore Avenue and Hotel Drive at UMD',
 'Bladensburg Waterfront Park',
 'Bowdoin Ave & Calvert Rd/ College Park Metro',
 'Bowdoin Ave & Calvert Rd/ College Park Station',
 'Capitol Heights Metro',
 'Chillum Rd & Riggs Rd / Riggs Plaza',
 'Crescent Rd & Ridge Rd',
 'Fleet St & Waterfront St',
 'Greenbelt Station Parkway',
 'Guilford Drive & Rowalt Drive / UMD',
 'Hyattsville Library / Adelphi Rd & Toledo Rd',
 "Largo Rd & Campus Way / Prince Georges's Comm Col",
 'Largo Town Center Metro',
 'National Harbor Carousel',
 'New Hampshire Ave & East-West Hwy',
 'Northwestern High School',
 'Oglethorpe St & 42nd Ave',
 'Oxon Hill Park & Ride',
 'Perry & 35th St',
 "Prince George's Plaza Metro",
 'Queens Chapel & Hamilton St',
 'Rhode Island Ave & 39th St / Brentwood Arts Exchange',
 'Rhode Island Avenue /Charles Armentrout Drive - Melrose Skate Park ',
 'Riggs Rd & East West Hwy',
 'Riverdale Park Town Center',
 'Roosevelt Center & Crescent Rd',
 'Southern Ave Metro',
 'Tanger Outlets',
 'The Mall at Prince Georges',
 'Walker Mill Road/ Walker Mill Regional Park ',
 'West Hyattsville Metro'])| prince_george["start_station_name"].isna()]

In [64]:
prince_george_fixed["start_station_name"].nunique()

35

In [48]:
prince_george_2024["start_station_name"].nunique()

73

take the wards gdf. Filter for dc.
take cabi maryland gdf. Filter for maryland
Join both

In [ ]:
dc_wards_gdf = dc_wards_gdf.drop(columns=["index_right"])

In [ ]:
maryland_gdf=maryland_gdf[["COUNTY","geometry"]]

In [ ]:
bllalbla

In [ ]:
# # Perform spatial join with the maryland gdf
# dc_maryland_gdf= gpd.sjoin(dc_wards_gdf, maryland_gdf, how="left", predicate="within")
#FIXME - i think this is were i messed up. Some of maryland stations are included inside dc. It should be a simple join, not spatial

In [28]:
dc_maryland_gdf[["NAME","COUNTY"]]

In [ ]:
# filtering out values that are not in maryland or dc
dc_maryland_gdf = dc_maryland_gdf[dc_maryland_gdf["NAME"].notnull()| dc_maryland_gdf["COUNTY"].notnull()]

In [ ]:
# new column to determine easy if it's dc or maryland
dc_maryland_gdf['area'] = np.where(dc_maryland_gdf['WARD'].notnull(), 'DC', 'Maryland')

/opt/anaconda3/envs/codeacademy/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
df_2022 = dc_maryland_gdf[dc_maryland_gdf["year"]==2022]
df_2023 = dc_maryland_gdf[dc_maryland_gdf["year"]==2023]
df_2024 = dc_maryland_gdf[dc_maryland_gdf["year"]==2024]

In [ ]:
# dc_maryland_gdf.to_csv("dc_maryland_gdf_full_updated.csv", index=False)

In [ ]:
#Total rides per year
print("total rides in 2022", len(df_2022))
print("total rides in 2023", len(df_2023))
print("total rides in 2024", len(df_2024))


NameError: name 'df_2022' is not defined

In [ ]:
# Rides per year per area
rides_DC = dc_maryland_gdf[dc_maryland_gdf["area"]=="DC"]
rides_maryland = dc_maryland_gdf[dc_maryland_gdf["area"]=="Maryland"]

: 

In [ ]:
#number of unique stations per year
print("unique stations in 2022", df_2022["start_station_name"].nunique())
print("unique stations in 2023", df_2023["start_station_name"].nunique())
print("unique stations in 2022", df_2024["start_station_name"].nunique())

unique stations in 2022 490
unique stations in 2023 564
unique stations in 2022 596
